In [0]:
# Install required modules
%pip install azure-core==1.27.1
%pip install azure-search-documents==11.4.0b3
%pip install azure-storage-blob==12.14.1
%pip install azure-identity==1.13.0b4

Python interpreter will be restarted.
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Not uninstalling typing-extensions at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-48678879-3b27-4b7c-91ca-4b0b0ad34dff
    Can't uninstall 'typing-extensions'. No files were found to uninstall.
Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
import os
import argparse
import glob
import html
import io
import re
#import time
#import base64
#from typing import Dict
#from pypdf import PdfReader, PdfWriter
from azure.identity import AzureDeveloperCliCredential, AzureCliCredential
from azure.core.credentials import AzureKeyCredential
from azure.storage.blob import BlobServiceClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import *
from azure.search.documents import SearchClient
#from azure.mgmt.msi import ManagedServiceIdentityClient

In [0]:
# Define connection info

# Get the service endpoint and API key from the environment
connection_string = "https://stfo2di6bg7kyzi.blob.core.windows.net"
container_name = "unzip" #"content"
from azure.identity import DefaultAzureCredential
default_creds = DefaultAzureCredential()

# Create a BlobServiceClient object using the connection string
blob_service_client = BlobServiceClient(account_url=connection_string, credential=default_creds)

In [0]:
blob_name = "TNO_archive.zip"
source_blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name )
# Check if the blob exists
blob_exists = source_blob_client.exists()

if blob_exists:
    print("Blob exists.")

Blob exists.


In [0]:
from io import BytesIO
import concurrent.futures
import zipfile
# Download the source blob into memory
downloaded_blob = source_blob_client.download_blob()
zip_data = downloaded_blob.content_as_bytes()
# Destination Blob container
destination_container_name = 'unzipped2'

# Function to extract and upload a file
def extract_and_upload(file_info):
    extracted_data = zip_ref.read(file_info.filename)
    destination_blob_name = file_info.filename
    destination_blob_client = blob_service_client.get_blob_client(container=destination_container_name, blob=destination_blob_name)
    destination_blob_client.upload_blob(extracted_data, overwrite=True)

# Extract the zip file in memory
with BytesIO(zip_data) as zip_stream:
    with zipfile.ZipFile(zip_stream, 'r') as zip_ref:
        # Create a ThreadPoolExecutor with a maximum of 10 worker threads
        with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
            # Iterate through each file in the zip and submit tasks to the executor
            futures = [executor.submit(extract_and_upload, file_info) for file_info in zip_ref.infolist()]
            
            # Wait for all tasks to complete
            concurrent.futures.wait(futures)

print("Zip file extracted and uploaded successfully.")

Zip file extracted and uploaded successfully.
